<a href="https://colab.research.google.com/github/zenikigai/Pengembangan_Machine_Learning_IDcamp2023/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

this lesson based on this https://github.com/zenikigai/Pengembangan_Machine_Learning_IDcamp2023/blob/main/Real_world_image_processing.ipynb

In [ ]:
!wget --no-check-certificate \
 https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip \
 -O /tmp/Chessman-image-dataset.zip

--2023-12-16 19:31:21--  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip [following]
--2023-12-16 19:31:21--  https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M   177MB/s    in 

In [ ]:
# using the same augmentation based on previous lesson
import os
import zipfile
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
local_zip = "/tmp/Chessman-image-dataset.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
train_dir = os.path.join("/tmp/Chessman-image-dataset/Chess")
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 20,
                                   zoom_range = 20,
                                   shear_range = 0.2,
                                   fill_mode = "nearest",
                                   validation_split = 0.1)

In [ ]:
# divide the dataset to be training data and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 8,
    class_mode = "categorical",
    subset = "training"
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 16,
    class_mode = "categorical",
    subset = "validation"
)

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


implement "Transfer Learning" here. The model we choose as the basic model of transfer learning is ResNet152V2. The ResNet152V2 model has as many as 152 layers and is available in keras library. We can use the features that have been learned by the model for our model to use.

To implement transfer learning is very easy as the code below. We only need to add 2 different lines of code. The first layer of our model is the model we use for transfer learning. We simply call the ResNet152V2 class and fill in the parameters as follows:

  Weight: For the weight parameter we fill in the value 'imagenet'. This means that we want to use the ResNet152V2 model that has been trained on the imagenet dataset. Imagenet is a giant database containing more than 14 million images.
  
  Include_top : this parameter is boolean. The point of this parameter is if we want to keep using the last layer / prediction layer from the resnet model. We fill in false because we are using a resnet model to predict the Chessman dataset instead of imagenet.
  
  Input_tensor : as the name implies this parameter specifies the size of the input.

In [ ]:
import tensorflow as tf
from keras.layers import Input, Flatten, Dense
from keras.applications import ResNet50, ResNet152V2
from keras.models import Sequential

model = Sequential([
    ResNet152V2(
         weights="imagenet",
         include_top=False,
         input_tensor=Input(shape=(150, 150, 3))
    ),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    Dense(6, activation="softmax")
])
model.layers[0].trainable = False

In [ ]:
# optimizer and loss
model.compile(optimizer = tf.optimizers.Adam(),
              loss="categorical_crossentropy",
              metrics = ["accuracy"])

In [ ]:
# train the model
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs = 50,
                    verbose = 1
                    )

Epoch 1/50
63/63 [==============================] - 164s 3s/step - loss: 1.9609 - accuracy: 0.2425 - val_loss: 2.0996 - val_accuracy: 0.2115
Epoch 2/50
63/63 [==============================] - 168s 3s/step - loss: 1.9602 - accuracy: 0.2445 - val_loss: 1.8603 - val_accuracy: 0.2885
Epoch 3/50
63/63 [==============================] - 170s 3s/step - loss: 1.9530 - accuracy: 0.2164 - val_loss: 2.0473 - val_accuracy: 0.1538
Epoch 4/50
63/63 [==============================] - 160s 3s/step - loss: 1.8164 - accuracy: 0.2164 - val_loss: 2.0566 - val_accuracy: 0.1923
Epoch 5/50
63/63 [==============================] - 170s 3s/step - loss: 1.8579 - accuracy: 0.2425 - val_loss: 1.9678 - val_accuracy: 0.1923
Epoch 6/50
63/63 [==============================] - 168s 3s/step - loss: 1.8121 - accuracy: 0.2505 - val_loss: 2.0065 - val_accuracy: 0.1154
Epoch 7/50
63/63 [==============================] - 170s 3s/step - loss: 1.8204 - accuracy: 0.2786 - val_loss: 1.9731 - val_accuracy: 0.2500
Epoch 8/50
63